### DuPont Mixture Design - Inverse Modeling

## Alison Shapiro, Sean Farrington, and Peter Osazuwa

***Machine Learning Techniques Used:*** 

Linear Regression

Gaussian Process Regression

## Script for Dual Annealing Optimization

#### Utilize predictions from the Gaussian process regression

Start by remaking the Gaussian process regression code

# Gaussian Process Regression


In [1]:
# Suppress Warnings
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" 

### Import new packages for GPR

In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import dual_annealing
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel as C, RBF, WhiteKernel
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error as mape, r2_score
import matplotlib.pyplot as plt

### Import data

In [3]:
file = 'DATA/training_inputs.xlsx'

df = pd.read_excel(file)

design = ['Powdered Additive','Base Resin A','Base Resin B','Stabilizer','Temperature','Screw Speed (RPM)']
performance = ['Toughness (J/m2)','Modulus (GPa)']

X = df[design]
y = df[performance]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(data=X,
                columns=design)
# print(X)

### Create model 

***Need two separate models for each output***

In [4]:
kernel = C() + RBF(length_scale=np.ones(X.shape[1]))

# Split to two models
reg_0 = GaussianProcessRegressor(kernel=kernel,random_state=1773).fit(X,y[performance[0]])
reg_1 = GaussianProcessRegressor(kernel=kernel,random_state=1773).fit(X,y[performance[1]])

y_pred_0 = reg_0.predict(X)
y_pred_1 = reg_1.predict(X)
y_pred_GPR = pd.DataFrame({performance[0]:y_pred_0,
                       performance[1]:y_pred_1
                       })

# Particle Swarm Routine

In this inverse model we will try to predict a data point from the testing set

The values for this datum is as follows:

- Powdered Additive: 0.32
- Base Resin A: 0.6
- Base Resin B: 0.05
- Stabilizer: 0.09
- Temperature: 406
- Screw Speed (RPM): 120


- Toughness (J/m2): 656
- Modulus (GPa): 5.1

In [5]:
import pyswarms as ps
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)

X_true = np.array([0.32,0.6,0.05,0.09,406,120])
y_true = np.array([656,5.1])

def CostFuncSlack(x,a,b):
    """
    Input 'x' is an array of unscaled variables. For this function they are:
    
    x = ['Powdered Additive','Base Resin A','Base Resin B','Temperature','Screw Speed']
    
    Notice that 'Stabilizer' is removed from this array, this must be accounted for so 
    scaling is done properly.
    
    Use 'y_0_target' and 'y_1_target' to assign the desired performance
    
    The cost function uses the slack variable approach to reduce
    dimensionality and ensure the composition is real.
    
    The barrier term here is a large penalty associated with the first three components 
    being greater than 1
    
    """
    y_0_target = a # Target toughness (J/m2)
    y_1_target = b # Target Modulus (GPa)
    
    sum_noslack = x[0]+x[1]+x[2]
    
    slack = 1 - (sum_noslack)
    
    x_full = np.insert(x,3,slack) # Insert slack into the fourth position
    
    x_scaled = scaler.transform(x_full.reshape(1,-1))
    
    y_0,std_0 = reg_0.predict(x_scaled,return_std=True)
    y_1,std_1 = reg_1.predict(x_scaled,return_std=True)
    
    var_0 = std_0**2/y_0_target # Normalized variance
    var_1 = std_1**2/y_1_target # Normalized variance
    
    y_0_penalty = ((y_0-y_0_target)/y_0_target)**2
    y_1_penalty = ((y_1-y_1_target)/y_1_target)**2
    
    barrier = 0
    if sum_noslack >= 1:
        barrier = 1e6
            
    loss = y_0_penalty + y_1_penalty + var_0 + var_1 + barrier
    return loss

# Dual annealing with slack variable

In [6]:
import time
start_time = time.time()

# Set-up Bounds
max_bound = [1,1,1,475,121]
min_bound = [0,0,0,380,79]

# Perform optimization
ret = dual_annealing(CostFuncSlack,
                    bounds=list(zip(min_bound,max_bound)),
                    args = (y_true[0],y_true[1]),
                    maxiter = 5_000,
                    seed = 1743)

# pos_unscaled = scaler.inverse_transform(pos.reshape(1,-1))
print(f'Best Cost = {ret.fun}')
print("--- %s seconds ---" % (time.time() - start_time))

Best Cost = 0.0003122615739444157
--- 47.560348987579346 seconds ---


In [7]:
ret

     fun: 0.0003122615739444157
 message: ['Maximum number of iteration reached']
    nfev: 124333
    nhev: 0
     nit: 5000
    njev: 12388
  status: 0
 success: True
       x: array([3.08899822e-01, 4.48104419e-01, 1.81938709e-01, 4.14337642e+02,
       8.66136936e+01])

In [8]:
pos = ret.x

Stabilizer = 1 - (pos[0]+pos[1]+pos[2])
    
pos = np.insert(pos,3,Stabilizer)

df = pd.DataFrame(data=pos.reshape(1,-1),
                index = ['Inverse Design'],
                columns = design)
df.loc['True Design'] = X_true
dif =  X_true - pos.reshape(-1)
df.loc['True - Inverse'] = dif
df.loc['Percent Difference'] = np.abs(dif/X_true)*100
df['Sum Composition'] = [np.sum(pos.reshape(-1)[0:4]),np.sum(X_true[0:4]),'N/A','N/A']

df.to_excel('ANALYSIS/DA_noInitialization_inputs.xlsx')

df

,Powdered Additive,Base Resin A,Base Resin B,Stabilizer,Temperature,Screw Speed (RPM),Sum Composition
Inverse Design,0.308900,0.448104,0.181939,0.061057,414.337642,86.613694,1.0
True Design,0.320000,0.600000,0.050000,0.090000,406.000000,120.000000,1.06
True - Inverse,0.011100,0.151896,-0.131939,0.028943,-8.337642,33.386306,N/A
Percent Difference,3.468805,25.315930,263.877418,32.158834,2.053607,27.821922,N/A


In [9]:
pos_scaled = scaler.transform(pos.reshape(1,-1))
y_pred_0 = reg_0.predict(pos_scaled)
y_pred_1 = reg_1.predict(pos_scaled)
y_pred_GPR = pd.DataFrame({performance[0]:y_pred_0,
                       performance[1]:y_pred_1
                       },index=['Inverse Performance'])

y_pred_GPR.loc['True Performance'] = y_true
dif = y_pred_GPR.loc['True Performance'] - y_pred_GPR.loc['Inverse Performance']
y_pred_GPR.loc['True - Inverse'] = dif
y_pred_GPR.loc['Percent Difference'] = np.abs(dif/y_true)*100

y_pred_GPR.to_excel('ANALYSIS/DA_noInitialization_outputs.xlsx')

y_pred_GPR

,Toughness (J/m2),Modulus (GPa)
Inverse Performance,657.453985,5.066972
True Performance,656.000000,5.100000
True - Inverse,-1.453985,0.033028
Percent Difference,0.221644,0.647599


# Dual annealing with initialization

In [10]:
# Initialization
init_pos = np.delete(X_true,3)

# Set-up Bounds
max_bound = [1,1,1,475,121]
min_bound = [0,0,0,380,79]

# Perform optimization
ret = dual_annealing(CostFuncSlack,
                    bounds=list(zip(min_bound,max_bound)),
                    args = (y_true[0],y_true[1]),
                    maxiter = 10_000,
                    seed = 1743,
                    x0 = init_pos)

# pos_unscaled = scaler.inverse_transform(pos.reshape(1,-1))
print(f'Best Cost = {ret.fun}')

Best Cost = 0.00031209085856768234


In [11]:
ret

     fun: 0.00031209085856768234
 message: ['Maximum number of iteration reached']
    nfev: 328021
    nhev: 0
     nit: 10000
    njev: 38002
  status: 0
 success: True
       x: array([3.08855114e-01, 4.48092994e-01, 1.82023536e-01, 4.14459256e+02,
       8.65483538e+01])

In [12]:
pos = ret.x

Stabilizer = 1 - (pos[0]+pos[1]+pos[2])
    
pos = np.insert(pos,3,Stabilizer)

df = pd.DataFrame(data=pos.reshape(1,-1),
                index = ['Inverse Design'],
                columns = design)
df.loc['True Design'] = X_true
dif =  X_true - pos.reshape(-1)
df.loc['True - Inverse'] = dif
df.loc['Percent Difference'] = np.abs(dif/X_true)*100
df['Sum Composition'] = [np.sum(pos.reshape(-1)[0:4]),np.sum(X_true[0:4]),'N/A','N/A']

df.to_excel('ANALYSIS/DA_yesInitialization_inputs.xlsx')

df

,Powdered Additive,Base Resin A,Base Resin B,Stabilizer,Temperature,Screw Speed (RPM),Sum Composition
Inverse Design,0.308855,0.448093,0.182024,0.061028,414.459256,86.548354,1.0
True Design,0.320000,0.600000,0.050000,0.090000,406.000000,120.000000,1.06
True - Inverse,0.011145,0.151907,-0.132024,0.028972,-8.459256,33.451646,N/A
Percent Difference,3.482777,25.317834,264.047072,32.190716,2.083561,27.876372,N/A


In [13]:
pos_scaled = scaler.transform(pos.reshape(1,-1))
y_pred_0 = reg_0.predict(pos_scaled)
y_pred_1 = reg_1.predict(pos_scaled)
y_pred_GPR = pd.DataFrame({performance[0]:y_pred_0,
                       performance[1]:y_pred_1
                       },index=['Inverse Performance'])

y_pred_GPR.loc['True Performance'] = y_true
dif = y_pred_GPR.loc['True Performance'] - y_pred_GPR.loc['Inverse Performance']
y_pred_GPR.loc['True - Inverse'] = dif
y_pred_GPR.loc['Percent Difference'] = np.abs(dif/y_true)*100

y_pred_GPR.to_excel('ANALYSIS/DA_yesInitialization_outputs.xlsx')

y_pred_GPR

,Toughness (J/m2),Modulus (GPa)
Inverse Performance,657.636987,5.067889
True Performance,656.000000,5.100000
True - Inverse,-1.636987,0.032111
Percent Difference,0.249541,0.629624
